# 🚀 Multi-Agent Text-to-SQL System (Agentic RAG)

Welcome! In this notebook, we will be build a fully autonomous AI agent capable of translating natural language into complex SQL queries, executing them against a relational database, catching its own errors, and returning business-ready insights.

We are utilizing **LangGraph** for the agentic workflow and the **Groq API** (Llama-3-70B) for lightning-fast inference.

## Step 1: Install Frameworks
We need LangChain and LangGraph for the AI orchestration, and Gradio to build our front-end user interface.

In [1]:
%%capture
pip install langchain langgraph sqlite3 pandas gradio

In [2]:
pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 6.7 MB/s eta 0:00:00


## Step 2: Initialize the LLM (Groq)
To make our agents think fast, we are using Groq.
When you run this cell, a secure text box will appear for you to paste your key.

In [3]:
import os
import getpass
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key:")
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0) # Temperature 0 ensures deterministic SQL generation

Enter your Groq API Key:··········


## Step 3: Build the Complex Relational Database
We are creating a realistic SQLite database named `ecommerce.db`. It contains four interconnected tables: Customers, Products, Orders, and OrderDetails. This simulates a real enterprise data environment.

In [4]:
import sqlite3
import random

print("Building E-Commerce Database...")
conn = sqlite3.connect('ecommerce.db')
c = conn.cursor()

# Drop tables if they exist (allows safe re-runs)
c.executescript('''
DROP TABLE IF EXISTS OrderDetails;
DROP TABLE IF EXISTS Orders;
DROP TABLE IF EXISTS Products;
DROP TABLE IF EXISTS Customers;
''')

# Create Tables with Primary and Foreign Keys
c.execute('''CREATE TABLE Customers (customer_id INTEGER PRIMARY KEY, name TEXT, segment TEXT, signup_date DATE)''')
c.execute('''CREATE TABLE Products (product_id INTEGER PRIMARY KEY, name TEXT, category TEXT, price REAL)''')
c.execute('''CREATE TABLE Orders (order_id INTEGER PRIMARY KEY, customer_id INTEGER, order_date DATE, status TEXT, FOREIGN KEY(customer_id) REFERENCES Customers(customer_id))''')
c.execute('''CREATE TABLE OrderDetails (detail_id INTEGER PRIMARY KEY, order_id INTEGER, product_id INTEGER, quantity INTEGER, FOREIGN KEY(order_id) REFERENCES Orders(order_id), FOREIGN KEY(product_id) REFERENCES Products(product_id))''')

# Generate Mock Data
categories = ['Electronics', 'Clothing', 'Home Goods', 'Sports']
segments = ['Retail', 'Wholesale', 'VIP']
statuses = ['Completed', 'Pending', 'Cancelled']

for i in range(1, 21):
    c.execute("INSERT INTO Products VALUES (?, ?, ?, ?)", (i, f'Product_A{i}', random.choice(categories), round(random.uniform(15.0, 300.0), 2)))

for i in range(1, 51):
    c.execute("INSERT INTO Customers VALUES (?, ?, ?, ?)", (i, f'Customer_{i}', random.choice(segments), '2025-06-15'))

for i in range(1, 101):
    c.execute("INSERT INTO Orders VALUES (?, ?, ?, ?)", (i, random.randint(1, 50), '2026-02-10', random.choice(statuses)))

for i in range(1, 251):
    c.execute("INSERT INTO OrderDetails VALUES (?, ?, ?, ?)", (i, random.randint(1, 100), random.randint(1, 20), random.randint(1, 5)))

conn.commit()
conn.close()
print("Database 'ecommerce.db' populated successfully!")

Building E-Commerce Database...
Database 'ecommerce.db' populated successfully!


## Step 4: Define the Agent State and Helper Functions
The agents need a shared memory space (the State) to pass data to each other. We also need a function that dynamically reads the database schema so the LLM knows what tables exist.

In [5]:
from typing import TypedDict

class AgentState(TypedDict):
    question: str
    schema: str
    sql_query: str
    db_result: str
    error: str
    iterations: int
    final_answer: str

def get_schema():
    conn = sqlite3.connect('ecommerce.db')
    c = conn.cursor()
    c.execute("SELECT sql FROM sqlite_master WHERE type='table';")
    schema = "\n".join([row[0] for row in c.fetchall() if row[0]])
    conn.close()
    return schema

## Step 5: Create the Multi-Agent Nodes
We are creating three specialized workers:
1. **SQL Writer:** Translates English to SQL using the schema.
2. **DB Executor:** Runs the SQL safely. If it fails, it captures the exact error.
3. **Data Analyst:** Translates the raw database numbers into a polished business response.

In [6]:
from langchain_core.prompts import ChatPromptTemplate

def sql_writer(state):
    prompt = ChatPromptTemplate.from_template(
        "You are an expert SQL developer. Write a valid SQLite query to answer the user's question.\n"
        "Schema:\n{schema}\n\nQuestion: {question}\n\n"
        "If you previously got an error, fix your code based on this message: {error}\n"
        "Return ONLY the SQL query. Do not include markdown formatting or explanations."
    )
    chain = prompt | llm
    response = chain.invoke({"schema": state["schema"], "question": state["question"], "error": state.get("error", "")})
    clean_sql = response.content.strip().replace("```sql", "").replace("```", "").strip()
    return {"sql_query": clean_sql, "iterations": state.get("iterations", 0) + 1}

def db_executor(state):
    query = state["sql_query"]
    try:
        conn = sqlite3.connect('ecommerce.db')
        c = conn.cursor()
        c.execute(query)
        result = c.fetchall()
        conn.close()
        return {"db_result": str(result), "error": ""}
    except Exception as e:
        return {"error": str(e), "db_result": ""}

def data_analyst(state):
    prompt = ChatPromptTemplate.from_template(
        "You are a data analyst. The user asked: {question}\n"
        "The database returned this raw data: {db_result}\n\n"
        "Write a clear, concise business answer based on this data. Do not mention SQL or databases."
    )
    chain = prompt | llm
    answer = chain.invoke({"question": state["question"], "db_result": state["db_result"]})
    return {"final_answer": answer.content}

## Step 6: Compile the Graph and Logic Router
This is the brain of the operation. We define a router that automatically sends the workflow *back* to the SQL Writer if the DB Executor hits an error (up to 3 times).

In [7]:
from langgraph.graph import StateGraph, END

def router(state):
    # If there is an error and we haven't tried too many times, self-correct.
    if state.get("error") and state.get("iterations") < 3:
        return "sql_writer"
    return "data_analyst"

workflow = StateGraph(AgentState)

workflow.add_node("sql_writer", sql_writer)
workflow.add_node("db_executor", db_executor)
workflow.add_node("data_analyst", data_analyst)

workflow.set_entry_point("sql_writer")
workflow.add_edge("sql_writer", "db_executor")
workflow.add_conditional_edges("db_executor", router)
workflow.add_edge("data_analyst", END)

app = workflow.compile()
print("Agentic Workflow Compiled!")

Agentic Workflow Compiled!


## Step 7: Launch the Web UI
We are wrapping our multi-agent system in a clean Gradio interface. This creates a public link you can share to demonstrate the project.

In [8]:
import gradio as gr

def run_agent(question):
    schema = get_schema()
    initial_state = {"question": question, "schema": schema, "iterations": 0, "error": ""}
    result = app.invoke(initial_state)
    return result["final_answer"], result["sql_query"]

interface = gr.Interface(
    fn=run_agent,
    inputs=gr.Textbox(lines=2, placeholder="e.g., Which product category has the highest average price?"),
    outputs=[
        gr.Textbox(label="Business Insights (Analyst Agent)"),
        gr.Textbox(label="Generated SQL (Writer Agent)")
    ],
    title="🤖 Autonomous Data Analyst Agent",
    description="Ask natural language questions about the enterprise database. The agent will autonomously write SQL, execute it, correct any syntax errors, and return business insights."
)

interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6d87d3618526a907d7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
